# Air hydrate segmentation

This notebook segments air hydrates included in microphotographs

![title](Segmentation_step.png)

#### Import the necessary packages

In [ ]:
import cv2
import skimage
###################################################
from skimage import color, filters, measure
from skimage import img_as_float, img_as_ubyte
from skimage.feature import canny
from skimage.morphology import closing, dilation, skeletonize, square, binary_erosion, disk, binary_closing
###################################################
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
###################################################
from scipy import ndimage as ndi
###################################################
import glob
import time
import os
import sys
from pathlib import Path

#### Version control

In [ ]:
from datetime import date 
today = date.today().isoformat()

print(f"Notebook last run in {today}")

In [ ]:
sys.version #Python

In [ ]:
cv2.__version__ #OpenCV

In [ ]:
skimage.__version__ #scikit-image

In [ ]:
pd.__version__ #Pandas

In [ ]:
np.__version__ #Numpy

In [ ]:
import scipy
scipy.__version__ #Numpy

In [ ]:
import matplotlib
matplotlib.__version__ #Numpy

#### Last edited:

29.10.2024

#### Set paths and name

In [ ]:
### Image series
series = "EDML C"

In [ ]:
### Segmentation category
category = "category_3"

In [ ]:
### Insert the directory where your images are saved
img_path = f".../{series}/{category}/"

In [ ]:
### Insert the directory where you want the results to be saved
save_path = f".../{series}/"

#### Create new paths

The next five lines create the necessary folders in the "save_path" directory where the results will be saved

In [ ]:
today = time.strftime("%d_%m_%Y")

In [ ]:
path_main = f"{save_path}/{today}/"
os.makedirs(path_main)

In [ ]:
path_category = f"{path_main}/{category}"
os.mkdir(path_category)

In [ ]:
path_data = f"{path_main}/{category}/data"
path_edges = f"{path_main}/{category}/edges"
path_segmented = f"{path_main}/{category}/segmented"
path_visual = f"{path_main}/{category}/visual_test"
path_final = f"{path_main}/{category}/seg_final"

In [ ]:
os.mkdir(path_data)
os.mkdir(path_edges)
os.mkdir(path_segmented)
os.mkdir(path_visual)
os.mkdir(path_final)

#### Get names of images in folder

In [ ]:
def load_image_names(img_path):
    img_names = []
    for filename in os.listdir(img_path):
        img_names.append(filename.replace(".tif", "")) ### change .tif to .png etc... depending on the type of your images
    return img_names

In [ ]:
names = load_image_names(img_path)

In [ ]:
names

## Segmentation routine

### Lists

Create lists for the saving of metadata

In [ ]:
seg_time=[] ### Segmentation timestamp
image_median=[] ### Median intensity
high_t=[] ### High value of hysteresis thresholding
filt2_count=[] ### Amount of objects filterted (experimental)

### Segmentation parameters

Adjust the segmentation parameters 

In [ ]:
### Gaussian blur
gauss=3
### Parameters for Canny edge detection
t_mult=1 ### High threshold multiplier for hysteresis thresholding
sigma=0.8 ### Canny sigma
low_threshold=0 ### Low threshold value for hysteresis thresholding
### Additional filters
size=100 ### area in pixel
aspect_ratio=5

In [ ]:
### Use this to only segment certain files in each category. Switch out "selected" for "names" in "Segmentation loop" below. 
selected = [names[1], names[3], names[5], names[8]]
selected

#### Measure process time

In [ ]:
start_all = time.time()

#### Segmentation loop

The following for-loop automatically loads and segments all the images included in the choosen directory (or selected names above).

In [ ]:
a=0
for img in names: ### Put "selected" instead of "names" here to segment above selected images.
    now=time.strftime("%d%m%Y-%H%M")
    seg_time.append(now)
    ##############Load the image##############
    original = cv2.imread(f"{img_path+img}.tif", cv2.IMREAD_GRAYSCALE) ### change .tif to .png etc... depending on the type of your images
    ##############Pre processing##############
    img_gauss = cv2.GaussianBlur(original,(gauss,gauss), cv2.BORDER_DEFAULT)
    ##############Edge detection Nr1##########
    med_intensity=np.median(original)
    image_median.append(med_intensity)
    high_threshold=med_intensity*t_mult
    high_t.append(high_threshold)
    ##########################################
    canny_1 = img_as_ubyte(canny(img_gauss,sigma,low_threshold,high_threshold))
    #cv2.imwrite(f"{path_edges}/{names[a]}_edges_S1.png", canny_1) ### Saves the first edge map
    ##############Fillholes Nr1###############
    closed_1=binary_closing(canny_1,footprint=square(1))
    fillholes_1=ndi.binary_fill_holes(closed_1)
    ##############Erode the image Nr1#########
    copy_1=fillholes_1.copy()
    eroded_1=binary_erosion(fillholes_1, out=copy_1) ### To prevent "tails" for the cases where an air hydrate is on a grain boundary.
    mask_1=~eroded_1 ### Inverse of eroded image
    image_1=img_as_ubyte(np.where(eroded_1 > 0, 255, eroded_1)) ### Set 1 to 255 for saving the image.
    ### Save SM1 ###
    cv2.imwrite(f"{path_segmented}/{names[a]}_segmented_S1.tif", image_1) ### IMPORTANT!!! image #S1, used for filtering step in a new jupyter notebook.
    ##############Edge detection Nr2##########
    canny_2 = img_as_ubyte(canny(img_gauss,sigma,low_threshold,high_threshold,mask=mask_1)) ### Uses S1 image as mask.
    ##############Fillholes Nr2###############
    fillholes_2=ndi.binary_fill_holes(canny_2)
    ##############Label the image#############
    img_label_2=measure.label(fillholes_2>0)
    ##############Regionproperties############
    props_2=measure.regionprops_table(img_label_2,original,
    properties=("label","area","coords","axis_minor_length","axis_major_length"))
    ##############Filter the data aspect-ratio#############
    ### Get objects that are bigger than "size" and have an AR bigger than "aspect_ratio".
    data_2=pd.DataFrame(props_2)
    data_filtered_2=data_2.copy()
    data_filtered_2["AR"] = (data_filtered_2["axis_major_length"])/(data_filtered_2["axis_minor_length"])
    data_filtered_2=data_filtered_2[data_filtered_2["area"]>size]
    data_filtered_2=data_filtered_2[data_filtered_2["AR"]>aspect_ratio]
    filter2_count = len(data_filtered_2) ### Count the amount of objects filtered in this step (experimental)
    filt2_count.append(filter2_count)
    ##############Create the mask#############
    ### Create a mask with objects that are bigger than "size" and have an AR bigger than "aspect_ratio".
    x_2=data_filtered_2["coords"].to_numpy(copy=True)
    mask_2=np.ones(original.shape, dtype=np.uint8)
    for y in x_2:
        for coord in y:
            mask_2[coord[0],coord[1]]=0
    save_mask_2=img_as_ubyte(np.where((mask_2==0),255,0))
    #cv2.imwrite(f"{path_edges}/{names[a]}_mask_S2.png", save_mask_2) ### Save image of filtered objects by "aspect_ratio".
    ###########################################
    edges_3=img_as_ubyte(np.where(((canny_2==0) | (mask_2==0)),0,1))
    save_edges_3 = np.where(edges_3> 0, 255, edges_3) ### Set 1 to 255 for saving the image.
    #cv2.imwrite(f"{path_edges}/{names[a]}_edges_S3.png", save_edges_3) ### Saves the second edge map
    ###########################################
    ##############Fillholes Nr3#########
    ### "Force"-close edges with a disk structuring element.
    dilated = dilation(edges_3, disk(3)) #square(5)
    skele = skeletonize(dilated)
    skele = img_as_ubyte(skele)
    fillholes_3 = img_as_ubyte(ndi.binary_fill_holes(skele))
    ##############Erode the image Nr3#########
    copy_3 = fillholes_3.copy()
    eroded_3 = binary_erosion(fillholes_3, out=copy_3) ### To prevent "tails" for the cases where an air hydrate is on a grain boundary.
    image_3 = np.where(eroded_3 > 0, 255, eroded_3)
    ### Save SM2 ###
    cv2.imwrite(f"{path_segmented}/{names[a]}_segmented_S3.tif", image_3) ### IMPORTANT!!! image #S3, used for filtering step in a new jupyter notebook.
    ###########################################
    r = img_as_ubyte(image_1)
    g = img_as_ubyte(save_mask_2)
    b = image_3
    bgr = cv2.merge((b,g,r))
    new = cv2.cvtColor(original,cv2.COLOR_GRAY2BGR)
    final = cv2.addWeighted(new,0.6,bgr,0.4,0)
    cv2.imwrite(f"{path_visual}/{names[a]}_visual_test.jpg",final) ### Saves an image (jpg) for a first visual assessment.
    a=a+1

#### Total time elapsed

Print the time needed to finish the segmentation step

In [ ]:
end_all = time.time()
time_all = (end_all-start_all)

In [ ]:
print("Elapsed time:", time_all, "seconds" )

## Create metadata

Saves the metadata as .csv file

In [ ]:
### Change "names" to "selected" as index if you chose to segment selected images.
metadata = pd.DataFrame(columns=['time', 'image_median','gauss','canny_sigma','canny_low','multiplier','canny_high',
                                 '>area','>aspect-ratio','filtered_AR'],index=names)

In [ ]:
metadata['time'] = seg_time
metadata['image_median'] = image_median
##
metadata.loc[:,'gauss'] = gauss
##
metadata.loc[:,'canny_sigma'] = sigma
metadata.loc[:,'canny_low'] = low_threshold
metadata.loc[:,'multiplier'] = t_mult
metadata['canny_high'] = high_t
##
metadata.loc[:,'>area'] = size
metadata.loc[:,'>aspect-ratio'] = aspect_ratio
metadata['filtered_AR'] = filt2_count

In [ ]:
metadata.head()

In [ ]:
metadata.to_csv(f"{path_main}/{category}/{category}_metadata_{today}.csv",sep=";")

# Finished